<a href="https://colab.research.google.com/github/TaahirBhorat/Zindi-Hack-Yassir/blob/master/Catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import model_selection, preprocessing, metrics
import matplotlib.pyplot as plt
!pip install -q catboost

     |████████████████████████████████| 64.8MB 61kB/s 


In [5]:
train = pd.read_csv('/content/drive/My Drive/Zindi/Train.csv')

In [27]:
train.head()


,ID,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,is_peak_traffic,Day_in_week,Day_in_year,Month
0,000FLWA8,3.258,36.777,3.003,36.718,39627,2784,0.200067,-0.052816,-0.050092,0.024168,0,2,338,12
1,000RGOAM,3.087,36.707,3.081,36.727,3918,576,0.034477,0.029167,0.027070,0.009644,0,1,344,12
2,001QSGIH,3.144,36.739,3.088,36.742,7265,526,0.089057,-0.006805,0.032986,-0.005816,0,5,327,11
3,002ACV6R,3.239,36.784,3.054,36.763,23350,3130,0.180617,-0.058447,-0.002421,-0.024345,0,6,335,12
4,0039Y7A8,2.912,36.707,3.207,36.698,36613,2138,-0.140080,0.041611,0.154813,0.029611,0,1,351,12


In [8]:
train.describe()

,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA
count,83924.000000,83924.000000,83924.000000,83924.000000,83924.000000,83924.000000
mean,3.052406,36.739358,3.056962,36.737732,13527.821410,1111.697762
std,0.096388,0.032074,0.100710,0.032781,9296.716006,563.565486
min,2.807000,36.589000,2.807000,36.596000,1.000000,1.000000
25%,2.994000,36.721000,2.995000,36.718000,6108.000000,701.000000
50%,3.046000,36.742000,3.049000,36.742000,11731.500000,1054.000000
75%,3.095000,36.760000,3.109000,36.760000,19369.000000,1456.000000
max,3.381000,36.820000,3.381000,36.819000,62028.000000,5238.000000


In [9]:
train.isnull().sum()

ID                 0
Timestamp          0
Origin_lat         0
Origin_lon         0
Destination_lat    0
Destination_lon    0
Trip_distance      0
ETA                0
dtype: int64

In [11]:
train.shape[0]

83924

In [19]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np

arr = np.vstack((train[['Origin_lat', 'Origin_lon']].values,
                    train[['Destination_lat', 'Destination_lon']].values))
sample_ind = np.random.permutation(len(arr))
kmeans =MiniBatchKMeans(n_clusters=90, batch_size=10000).fit(arr[sample_ind])

In [ ]:
train.loc[:, 'pickup_cluster'] = kmeans.predict(train[['Origin_lat', 'Origin_lon']])

train.loc[:, 'dropoff_cluster'] = kmeans.predict(train[['Destination_lat', 'Destination_lon']])

In [20]:
arr = np.vstack((train[['Origin_lat', 'Origin_lon']].values,
                    train[['Destination_lat', 'Destination_lon']].values))

from sklearn.decomposition import PCA
pca = PCA().fit(arr)
train['pickup_pca0'] = pca.transform(train[['Origin_lat', 'Origin_lon']])[:, 0]
train['pickup_pca1'] = pca.transform(train[['Origin_lat', 'Origin_lon']])[:, 1]
train['dropoff_pca0'] = pca.transform(train[['Destination_lat', 'Destination_lon']])[:, 0]
train['dropoff_pca1'] = pca.transform(train[['Destination_lat', 'Destination_lon']])[:, 1]

In [26]:
StartTime = pd.to_datetime(train['Timestamp'], infer_datetime_format=True)

KeyError: ignored

In [25]:
train['is_peak_traffic'] = [1 if (5<i<9 or 15<i<20) else 0 for i in StartTime.dt.hour]
train['Day_in_week'] = StartTime.dt.dayofweek
train['Day_in_year'] = StartTime.dt.dayofyear
train['Month'] = StartTime.dt.month
train = train.drop('Timestamp', axis=1)
train = train.drop('Timestamp', axis=1)


KeyError: ignored

In [28]:
train.head()

,ID,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,is_peak_traffic,Day_in_week,Day_in_year,Month
0,000FLWA8,3.258,36.777,3.003,36.718,39627,2784,0.200067,-0.052816,-0.050092,0.024168,0,2,338,12
1,000RGOAM,3.087,36.707,3.081,36.727,3918,576,0.034477,0.029167,0.027070,0.009644,0,1,344,12
2,001QSGIH,3.144,36.739,3.088,36.742,7265,526,0.089057,-0.006805,0.032986,-0.005816,0,5,327,11
3,002ACV6R,3.239,36.784,3.054,36.763,23350,3130,0.180617,-0.058447,-0.002421,-0.024345,0,6,335,12
4,0039Y7A8,2.912,36.707,3.207,36.698,36613,2138,-0.140080,0.041611,0.154813,0.029611,0,1,351,12


In [32]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

X = train.drop('ETA', axis=1)
y = train['ETA']

print('splitting into test, validation and training sets')
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.01)
X_val, X_test, y_val, y_test = train_test_split(X, y, shuffle=True, test_size=0.5)

model = CatBoostRegressor(
    loss_function='RMSE',
    iterations=20000,
    learning_rate=1.0,
    task_type='GPU'
)

print('training catboost model')
model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    verbose=200)

ModuleNotFoundError: ignored